In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from ast import literal_eval
from collections import defaultdict
%matplotlib inline
import random
from sklearn.preprocessing import MinMaxScaler

In [4]:
def FindBestSellers(Prod,data):
    df = data.loc[data['Product'] == Prod]
    df = df[['Seller', 'Price','FeedBack_Rating', 'Packaging_Rating', 'Courier_Rating','Product_Rating']]

    avg_price = pd.DataFrame(df.groupby('Seller')['Price'].mean().sort_values()).reset_index()
    avg_rating_count = pd.DataFrame(df.groupby('Seller')['FeedBack_Rating'].count().sort_values(ascending=False)).reset_index()
    avg_buyer_feedback = pd.DataFrame(df.groupby('Seller')['FeedBack_Rating'].mean().sort_values(ascending=False)).reset_index()
    avg_courier_rating = pd.DataFrame(df.groupby('Seller')['Courier_Rating'].mean().sort_values()).reset_index()
    avg_product_rating = pd.DataFrame(df.groupby('Seller')['Product_Rating'].mean().sort_values()).reset_index()
    avg_packaging_rating = pd.DataFrame(df.groupby('Seller')['Packaging_Rating'].mean().sort_values()).reset_index()

    new_Frame1 = pd.merge(avg_rating_count,avg_buyer_feedback,on='Seller', how='outer')
    new_Frame2 = pd.merge(avg_courier_rating,avg_product_rating,on='Seller', how='outer')
    new_Frame3 = pd.merge(avg_price,avg_packaging_rating,on='Seller', how='outer')

    d1 = pd.merge(new_Frame1,new_Frame2,on='Seller', how='outer')
    dataSet = pd.merge(d1,new_Frame3,on='Seller', how='outer')

    dataSet = dataSet.rename(columns = {'FeedBack_Rating_x':'Count','FeedBack_Rating_y':'FeedBack_Rating'})
    
    data = dataSet[['Price','Count','FeedBack_Rating','Courier_Rating','Product_Rating','Packaging_Rating']]
    scaler = MinMaxScaler(feature_range=(0, 5))
    scaler.fit(data)
    df = pd.DataFrame(scaler.transform(data))
    df[0] = 5  - df[0]
    
    lis = ['Price','Count','FeedBack_Rating','Courier_Rating','Product_Rating','Packaging_Rating']
    corr = df.corr()
    for i in np.arange(0,6):
        sr = lis[i]+'_Score'
        corrList = corr[i]
        dataSet[sr] = (corrList[0]*df[0]) + (corrList[1]*df[1]) + (corrList[2]*df[2]) + (corrList[3]*df[3]) +(corrList[4]*df[4]) + (corrList[5]*df[5])
    
    PS = dataSet.sort_values(by='Price_Score',ascending=False).head(1)['Seller'].astype(str)
    CS = dataSet.sort_values(by='Count_Score',ascending=False).head(1)['Seller'].astype(str)
    BFS = dataSet.sort_values(by='FeedBack_Rating_Score',ascending=False).head(1)['Seller'].astype(str)
    CRS = dataSet.sort_values(by='Courier_Rating_Score',ascending=False).head(1)['Seller'].astype(str)
    PRS = dataSet.sort_values(by='Product_Rating_Score',ascending=False).head(1)['Seller'].astype(str)
    PARS = dataSet.sort_values(by='Packaging_Rating_Score',ascending=False).head(1)['Seller'].astype(str)
      
    dataSet['ALLS'] = df[0] + df[1] + df[2] + df[3] + df[4] +df[5]
    ALLS = dataSet.sort_values(by='ALLS',ascending=False).head(1)['Seller'].astype(str)
    
    drp = {}
    drp['Product'] = Prod
    drp['Price'] = PS
    drp['Count'] = CS
    drp['FeedBack_Rating'] = BFS
    drp['Courier_Rating'] = CRS
    drp['Product_Rating'] = PRS
    drp['Packaging_Rating'] = PARS
    drp['ALL OVER'] = ALLS
    
    return drp

In [5]:
dr = pd.unique(data['Product'])
fadr = pd.DataFrame()
drro = pd.DataFrame()
for item in dr:
    drrt = FindBestSellers(item,data)
    df = pd.DataFrame(drrt)
    df.fillna(value='HELLO',inplace=True)
    drrp = pd.DataFrame(df.replace('HELLO','').groupby('Product',as_index=False).agg(''.join))
    drro =  pd.concat([drro,drrp],axis=0)

NameError: name 'data' is not defined

In [6]:
drro.to_csv('D:/DST/BestSellers_Product_Wise.csv')

NameError: name 'drro' is not defined

In [28]:
dataFrame = pd.read_csv('D:/DST/BestSellers_Product_Wise.csv')
dataFrame = dataFrame[['Product', 'Price', 'Count', 'FeedBack_Rating','Courier_Rating', 'Product_Rating', 'Packaging_Rating', 'ALL OVER']]
data = pd.read_csv('D:/DST/FinalData.csv')
data = data[['Product', 'Brand']]
dataFrame = dataFrame.merge(data,how='left')
dataFrame = dataFrame.drop_duplicates(keep="first")

dr = pd.DataFrame(dataFrame.groupby(['Brand','Price'])['Price'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Price'})
dr = dr[['Brand','Price']]
Price = dr

dr = pd.DataFrame(dataFrame.groupby(['Brand','Count'])['Count'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Count'})
dr = dr[['Brand','Count']]
Count = dr

dr = pd.DataFrame(dataFrame.groupby(['Brand','FeedBack_Rating'])['FeedBack_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'FeedBack_Rating'})
dr = dr[['Brand','FeedBack_Rating']]
FeedBack_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['Brand','Courier_Rating'])['Courier_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Courier_Rating'})
dr = dr[['Brand','Courier_Rating']]
Courier_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['Brand','Product_Rating'])['Product_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Product_Rating'})
dr = dr[['Brand','Product_Rating']]
Product_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['Brand','Packaging_Rating'])['Packaging_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Packaging_Rating'})
dr = dr[['Brand','Packaging_Rating']]
Packaging_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['Brand','ALL OVER'])['ALL OVER'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'ALL OVER'})
dr = dr[['Brand','ALL OVER']]
ALL_OVER = dr

f1 = pd.merge(Packaging_Rating, ALL_OVER , on=['Brand'])
f2 = pd.merge(Price, Count, on=['Brand'])
f3 = pd.merge(Courier_Rating, FeedBack_Rating , on=['Brand'])

f12 =  pd.merge(f1,f2 , on=['Brand'])
f34 =  pd.merge(f3,Product_Rating , on=['Brand'])

finSet = pd.merge(f12,f34 , on=['Brand'])

finSet.to_csv('D:/DST/BestSellers_Brand_Wise.csv')

In [30]:
dataFrame = pd.read_csv('D:/DST/BestSellers_Product_Wise.csv')
dataFrame = dataFrame[['Product', 'Price', 'Count', 'FeedBack_Rating','Courier_Rating', 'Product_Rating', 'Packaging_Rating', 'ALL OVER']]
data = pd.read_csv('D:/DST/FinalData.csv')
data = data[['Product', 'MultiLink']]
dataFrame = dataFrame.merge(data,how='left')
dataFrame = dataFrame.drop_duplicates(keep="first")

dr = pd.DataFrame(dataFrame.groupby(['MultiLink','Price'])['Price'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Price'})
dr = dr[['MultiLink','Price']]
Price = dr

dr = pd.DataFrame(dataFrame.groupby(['MultiLink','Count'])['Count'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Count'})
dr = dr[['MultiLink','Count']]
Count = dr

dr = pd.DataFrame(dataFrame.groupby(['MultiLink','FeedBack_Rating'])['FeedBack_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'FeedBack_Rating'})
dr = dr[['MultiLink','FeedBack_Rating']]
FeedBack_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['MultiLink','Courier_Rating'])['Courier_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Courier_Rating'})
dr = dr[['MultiLink','Courier_Rating']]
Courier_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['MultiLink','Product_Rating'])['Product_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Product_Rating'})
dr = dr[['MultiLink','Product_Rating']]
Product_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['MultiLink','Packaging_Rating'])['Packaging_Rating'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'Packaging_Rating'})
dr = dr[['MultiLink','Packaging_Rating']]
Packaging_Rating = dr

dr = pd.DataFrame(dataFrame.groupby(['MultiLink','ALL OVER'])['ALL OVER'].count())
dr = dr.unstack(level=0).T
dr = pd.DataFrame(dr.idxmax(axis=1)).reset_index()
dr = dr.rename(columns={0:'ALL OVER'})
dr = dr[['MultiLink','ALL OVER']]
ALL_OVER = dr

f1 = pd.merge(Packaging_Rating, ALL_OVER , on=['MultiLink'])
f2 = pd.merge(Price, Count, on=['MultiLink'])
f3 = pd.merge(Courier_Rating, FeedBack_Rating , on=['MultiLink'])

f12 =  pd.merge(f1,f2 , on=['MultiLink'])
f34 =  pd.merge(f3,Product_Rating , on=['MultiLink'])

finSet = pd.merge(f12,f34 , on=['MultiLink'])

finSet.to_csv('D:/DST/BestSellers_Category_Wise.csv')